In [1]:
import pandas as pd

In [2]:
orig = pd.read_csv('tikhonov_train.txt', sep='\t', header=0, names=['word', 'morphemes'])

In [3]:
orig

,word,morphemes
0,грузовик,груз:ROOT/ов:SUFF/ик:SUFF
1,краниометрический,крани:ROOT/о:LINK/метр:ROOT/ическ:SUFF/ий:END
2,зернить,зерн:ROOT/и:SUFF/ть:SUFF
3,индуистский,инду:ROOT/ист:SUFF/ск:SUFF/ий:END
4,набело,на:PREF/бел:ROOT/о:END
...,...,...
76691,кампешевый,кампеш:ROOT/ев:SUFF/ый:END
76692,последний,последн:ROOT/ий:END
76693,шататься,шат:ROOT/а:SUFF/ть:SUFF/ся:POSTFIX
76694,шахиншахский,шах:ROOT/ин:ROOT/шах:ROOT/ск:SUFF/ий:END


In [4]:
def count_overlapping_substrings(haystack, needle):
    count = 0
    i = -1
    while True:
        i = haystack.find(needle, i+1)
        if i == -1:
            return count
        count += 1

In [5]:
def prepare_data(row):
    return count_overlapping_substrings(row['morphemes'], 'ROOT')

In [6]:
orig['result'] = orig.apply(prepare_data, axis=1)

In [7]:
orig[orig['result']>1].describe()

,result
count,15053.000000
mean,2.100512
std,0.327348
min,2.000000
25%,2.000000
50%,2.000000
75%,2.000000
max,6.000000


In [8]:
orig[orig['result']>1]['word']

1        краниометрический
6              полутоновый
9              психологист
16              криводушие
19            красногрудый
               ...        
76683     лжесвидетельница
76686               пухоед
76688            двугорбый
76694         шахиншахский
76695              роспуск
Name: word, Length: 15053, dtype: object

In [41]:
test = pd.read_csv('ds_common_words_utf_8.csv')

In [45]:
test

,word,root
0,белить,бел
1,белобрысый,бел
2,белозубый,бел
3,белок,бел
4,белолицый,бел
...,...,...
1030,крашеный,крас
1031,окрас,крас
1032,подкрасить,крас
1033,раскраска,крас


In [43]:
def prepaire_root(row):
    root_split = row['root'].split('_')[0].split('(')
    base_root = root_split[0]
    if len(root_split) > 1:
        end_root = root_split[1][:-1]
        other_root = f'{base_root[:-len(end_root)]}{end_root}'
        if other_root in row['word']:
            return other_root
    return base_root

In [44]:
test['root'] = test.apply(prepaire_root, axis=1)

In [28]:
test.to_csv('prepaired_ds_common.csv', index=False)

In [61]:
merged = orig.merge(test,on=['word'])

In [62]:
def parse_morphemes(row):
    morphemes = row['morphemes'].split('/')
    roots = []
    for morph in morphemes:
        part, type_morph = morph.split(':')
        if type_morph == 'ROOT':
            if part.endswith('ь'):
                part = part[:-1]
            roots.append(part)
            
    return " ".join(roots)
    

In [64]:
merged[merged['result'] > 1]

,word,morphemes,result,root
1,красногрудый,красн:ROOT/о:LINK/груд:ROOT/ый:END,2,крас
2,снегозадержание,снег:ROOT/о:LINK/за:PREF/держ:ROOT/ани:SUFF/е:END,2,снег
6,хлебопродукты,хлеб:ROOT/о:LINK/продукт:ROOT/ы:END,2,хлеб
12,раболепие,раб:ROOT/о:LINK/леп:ROOT/и:SUFF/е:END,2,раб
31,белорыбица,бел:ROOT/о:LINK/рыб:ROOT/иц:SUFF/а:END,2,бел
...,...,...,...,...
662,горемыка,гор:ROOT/е:LINK/мык:ROOT/а:END,2,гор
671,медонос,мед:ROOT/о:LINK/нос:ROOT,2,мед
683,белохвостый,бел:ROOT/о:LINK/хвост:ROOT/ый:END,2,бел
688,домохозяйка,дом:ROOT/о:LINK/хозяй:ROOT/к:SUFF/а:END,2,дом


In [55]:
merged['root']=merged.apply(parse_morphemes, axis=1)

In [56]:
for ind, r in merged.iterrows():
    if r['root'] not in r['roots']:
        print(ind, r['word'], r['roots'],r['root'])

1 красногрудый ['красн', 'груд'] красн груд
2 снегозадержание ['снег', 'держ'] снег держ
6 хлебопродукты ['хлеб', 'продукт'] хлеб продукт
12 раболепие ['раб', 'леп'] раб леп
31 белорыбица ['бел', 'рыб'] бел рыб
34 водопад ['вод', 'пад'] вод пад
41 зверовод ['звер', 'вод'] звер вод
46 садовод ['сад', 'вод'] сад вод
48 спортзал ['спорт', 'зал'] спорт зал
54 листоед ['лист', 'ед'] лист ед
55 овощемоечный ['овощ', 'мо'] овощ мо
57 домосед ['дом', 'сед'] дом сед
58 словесник ['слов', 'ес'] слов ес
59 домостроение ['дом', 'стро'] дом стро
61 судовладелец ['суд', 'влад'] суд влад
64 травополье ['трав', 'пол'] трав пол
70 куровод ['кур', 'вод'] кур вод
71 жаропрочный ['жар', 'прочн'] жар прочн
74 лесостепь ['лес', 'степ'] лес степ
79 игротека ['игр', 'тек'] игр тек
89 ракета-носитель ['ракет', 'нос'] ракет нос
92 хлебозавод ['хлеб', 'завод'] хлеб завод
94 камнеломка ['камн', 'лом'] камн лом
96 утконос ['утк', 'нос'] утк нос
100 землекоп ['земл', 'коп'] земл коп
101 спортплощадка ['спорт', 'пло

In [57]:
test[test['word'].isin(orig['word'])]

,word,root
0,белить,бел
2,белозубый,бел
4,белолицый,бел
5,белолобый,бел
6,белоручка,бел
...,...,...
1030,крашеный,крас
1031,окрас,крас
1032,подкрасить,крас
1033,раскраска,крас


In [58]:
result = merged

In [59]:
del result['morphemes']
del result['roots']
del result['result']
result

,word,root
0,грузовик,груз
1,красногрудый,красн груд
2,снегозадержание,снег держ
3,воробышек,вороб
4,игровой,игр
...,...,...
692,конина,кон
693,курник,кур
694,березина,берез
695,судопроизводство,суд производ


In [65]:
result

,word,root
0,грузовик,груз
1,красногрудый,красн груд
2,снегозадержание,снег держ
3,воробышек,вороб
4,игровой,игр
...,...,...
692,конина,кон
693,курник,кур
694,березина,берез
695,судопроизводство,суд производ


In [68]:
test[test['word'].isin(result['word'])]

,word,root
0,белить,бел
2,белозубый,бел
4,белолицый,бел
5,белолобый,бел
6,белоручка,бел
...,...,...
1030,крашеный,крас
1031,окрас,крас
1032,подкрасить,крас
1033,раскраска,крас


In [70]:
for ind, r in result.iterrows():
    test.loc[test['word']==r['word'], 'root'] = r['root']

In [76]:
test.to_csv('ds_common_words_utf_8_for_test.csv', index=None)

In [74]:
test = test.sort_values(by=['word'])

In [75]:
test

,word,root
64,азбука,азбук
65,азбучный,азбуч
244,бездомный,дом
855,бездорожье,дорож
636,безухий,ух
...,...,...
586,цветовод,цвет вод
587,цветочек,цвет
588,цветочный,цвет
589,цветущий,цвет
